# Bert with tweet_eval
fine tune a bert model with tweet_eval for sentiment classification

In [5]:
from datasets import load_dataset
import pandas as pd
import numpy as np

import ktrain
from ktrain import text

## 1. Import data

In [10]:
# download dataset from hungging face

dataset = load_dataset("tweet_eval", "sentiment")
dataset

Found cached dataset tweet_eval (C:/Users/Splute/.cache/huggingface/datasets/tweet_eval/sentiment/1.1.0/12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 45615
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 12284
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})

In [9]:
data_train = pd.DataFrame(dataset['train'])
data_test = pd.DataFrame(dataset['test'])

print(data_train)
print(data_test)

size of training set: 45615
size of validation set: 12284
1    26610
2    20224
0    11065
Name: label, dtype: int64


,text,label
0,"""QT @user In the original draft of the 7th boo...",2
1,"""Ben Smith / Smith (concussion) remains out of...",1
2,Sorry bout the stream last night I crashed out...,1
3,Chase Headley's RBI double in the 8th inning o...,1
4,@user Alciato: Bee will invest 150 million in ...,2
5,@user LIT MY MUM 'Kerry the louboutins I wonde...,2
6,"""\"""""""" SOUL TRAIN\"""""""" OCT 27 HALLOWEEN SPECIA...",2
7,So disappointed in wwe summerslam! I want to s...,0
8,"""This is the last Sunday w/o football .....,NF...",2
9,@user @user CENA & AJ sitting in a tree K-I-S-...,1


In [ ]:

X_train = data_train.text.tolist()
X_test = data_test.text.tolist()

y_train = data_train.label.tolist()
y_test = data_test.label.tolist()

data = data_train.append(data_test, ignore_index=True)

class_names = ['negtive', 'neutral', 'positive']

print('size of training set: %s' % (len(data_train['text'])))
print('size of validation set: %s' % (len(data_test['text'])))
print(data.label.value_counts())

data.head(10)

## 2. Data preprocessing

In [ ]:
(x_train,  y_train), (x_test, y_test), preproc = text.texts_from_array(x_train=X_train, y_train=y_train,
                                                                       x_test=X_test, y_test=y_test,
                                                                       class_names=class_names,
                                                                       preprocess_mode='bert',
                                                                       maxlen=350,
                                                                       max_features=35000)